In [3]:
# Header
from nupack import *
import numpy as np
config.parallelism = True

Model1 = Model(material='dna', celsius=29, sodium=0.1, magnesium=0.0125) #Define model used for NUPACK calculations

In [4]:
# Import strands
Strands_list = []
Interfering = open("N25.txt")
for line in Interfering:
    Strands_list.append(line.rstrip())
Interfering.close()
Nlength = 25 #Number of bases of each interfering strand (fixed in our case)

In [5]:
# Define strands/complexes and tasks 
Invader_Strand = Strand('CGGAATAAGGCAAGATAAGAACAGATACAT', name='Invader')
Interfering_free_energy_list = []
results = open("Results25.txt", 'a')
for i in range(len(Strands_list)):
    Interfering_Strand = Strand(Strands_list[i], name ='Interfering_Strand')
    Interfering_Invader_Complex = Complex([Interfering_Strand, Invader_Strand], name='Interfering_Invader_Complex')
    Interfering_Complex = Complex([Interfering_Strand], name='Interfering_Complex')
    Invader_Complex = Complex([Invader_Strand], name='Invader_Complex')
    InteractionTube = Tube({Invader_Strand: 20e-9, Interfering_Strand: 1e-6}, complexes=SetSpec(include=[Interfering_Invader_Complex, Interfering_Complex, Invader_Complex]), name='InteractionTube')
    my_results = tube_analysis([InteractionTube], model=Model1, compute=['mfe'])
    Interfering_free_energy_list.append(str(my_results[Interfering_Complex].free_energy))
    results.write(str(Interfering_Strand) + '\n' +str(my_results[Interfering_Invader_Complex].free_energy) +'\n'+ str(my_results[Interfering_Invader_Complex].mfe[0].structure) +'\n'+'\n')
results.close()

In [9]:
# Calculate Energies
Results_list = []

results = open("Results25.txt")
for line in results:
    Results_list.append(line.rstrip())
results.close()

Energies_list=[]

Complexes_ddG = open("Complex_ddG25.txt", 'a')

for i in range(len(Results_list[1::4])):
    Energies_list.append(float(Results_list[1::4][i])-float(Interfering_free_energy_list[i])-my_results[Invader_Complex].free_energy)
    Complexes_ddG.write(str("{:.2f}".format(Energies_list[i]))+'\n')
Complexes_ddG.close()

In [10]:
# Open the file and read in the numbers
with open('Complex_ddG25.txt', 'r') as f:
    numbers = [float(line.strip()) for line in f]

# Sort numbers
numbers.sort()

with open('N25_Energies_sorted.txt', 'w') as f:
    for number in numbers:
        f.write(str(number) + '\n')